In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [2]:
#Funcion que limpia los datos, quedandose con el nivel de educacion mas alto de casa id de los postulantes.
def education_cleaning(df):
    education_dic= {'Secundario': 1, 'Universitario':3,'Terciario/Técnico':2, 'Otro':0, 'Posgrado':4, 'Master': 5, 'Doctorado': 6}
    df['nombre_sort'] = df['nombre'].map(education_dic)
    df = df.sort_values(by='nombre_sort', ascending=False)
    df = df.drop_duplicates(subset='idpostulante',keep='first')
    return df

In [3]:
def datetime(date,format_string):
    return pd.to_datetime(date,format= format_string,errors='coerce')

In [4]:
merge_postulants_ads = pd.read_csv('avisos_postulantes.csv')

/home/olix/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
merge_postulants_ads['se_postulo'] = 1

In [6]:
merge_postulants_ads_NOTS = pd.read_csv('avisos_no_postulantes.csv')

/home/olix/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
#Concateno las postulaciones con las NO postulaciones

frames = [merge_postulants_ads,merge_postulants_ads_NOTS]
total = pd.concat(frames, ignore_index = True)
total.drop_duplicates(subset=['idpostulante','idaviso'], keep = 'first',inplace = True)
total.reset_index(inplace = True,drop = True)

MemoryError: 

In [ ]:
#Aca empezaria a ver que datos voy a usar para la parte de machine learning y entonces saco
#los datos que no voy a usar y mapeo los datos segun sea necesario.SERIA MEJOR QUE ESTE EN OTRO NOTEBOOK

In [ ]:
#Aca empiezo a sacar las columnas que no iba a usar para el arbol de decision
total.drop(columns = ['ciudad','mapacalle','nombre_sort','idpais','descripcion','fechapostulacion','titulo'],inplace = True)

In [1]:
#Paso a datetime la parte de fecha de nacimiento
total['edadPostulante'] = datetime(total['fechanacimiento'],'%Y-%m-%d')

NameError: name 'datetime' is not defined

In [2]:
#Calculo la edad de cada postulante
total['edadPostulante'] = total['edadPostulante'].map(lambda x: 2018 - x.year)
#Aca tengo que poner lo de rellenar las edades con su promedio

NameError: name 'total' is not defined

In [ ]:
age_std = total['edadPostulante'].std()
age_null_count = total['edadPostulante'].isnull().sum()
age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
 
conValoresNulos = np.isnan(total['edadPostulante'])
 
total.loc[np.isnan(total['edadPostulante']), 'edadPostulante'] = age_null_random_list
total['edadPostulante'] = total['edadPostulante'].astype(int)
print("Edad Promedio: " + str(age_avg))
print("Desvió Std Edad: " + str(age_std))
print("Intervalo para asignar edad aleatoria: " + str(int(age_avg - age_std)) + " a " + str(int(age_avg + age_std)))

In [ ]:
total.drop(columns = ['fechanacimiento'],inplace = True)

In [ ]:
# estado Mapping 
total['estadoEncoded'] = total['estado'].map( {'En Curso': 0, 'Abandonado': 2, 'Graduado': 1, '': 0} ).astype(int)
# nivelEducativo Mapping 
total['nivelEducativoEncoded'] = total['nombre'].map( {'Secundario': 1, 'Universitario':3,'Terciario/Técnico':2, 'Otro':0, 'Posgrado':4, 'Master': 5, 'Doctorado': 6}
                                       ).astype(int)
# tipoDeTrabajo Mapping 
total['tipoDeTrabajoEncoded'] = total['tipo_de_trabajo'].map( {'Full-time':0, 'Part-time':1, 'Teletrabajo':2, 'Por Horas': 3, 
                                                              'Pasantia': 4, 'Temporario': 5, 'Por Contrato': 6, 
                                                              'Fines de Semana': 7, 'Primer empleo': 8} ).astype(int)
 

    
# Nivel de trabajo Mapping
total['nivelLaboralEncoded'] = total['nivel_laboral'].map( {'Senior / Semi-Senior':0, 'Junior':1, 'Otro':2, 'Jefe / Supervisor / Responsable': 3, 
                                                              'Gerencia / Alta Gerencia / Dirección': 4} ).astype(int)


#Sexo Mapping

total['sexoEncoded'] = total['sexo'].map({'FEM':0, 'MASC':1, 'NO_DECLARA':2}).astype(int)


# Mapping edad del postulante
total.loc[ total['edadPostulante'] <= 21, 'edadEncoded']                         = 0
total.loc[(total['edadPostulante'] > 21) & (total['edadPostulante'] <= 26), 'edadEncoded'] = 1
total.loc[(total['edadPostulante'] > 26) & (total['edadPostulante'] <= 30), 'edadEncoded'] = 2
total.loc[(total['edadPostulante'] > 30) & (total['edadPostulante'] <= 40), 'edadEncoded'] = 3
total.loc[ total['edadPostulante'] > 40, 'edadEncoded'] = 4

In [ ]:
drop_elements = ['idaviso','idpostulante','nombre','tipo_de_trabajo','estado','sexo','edadPostulante','denominacion_empresa','nivel_laboral','nombre_area','nombre_zona']
postulations_encoded = total.drop(drop_elements, axis = 1)